In [ ]:
from arc.data import make_lexicon_generator, sample_min_overlap_lexicon, word_overlap_matrix

from arc.types import from_json, Word

words = from_json("words_200_filtered.json")

lexicon_generator_1 = make_lexicon_generator(words, n_words=4, max_overlap=1, max_yields=1000)
lexicon_generator_2 = sample_min_overlap_lexicon(words, word_overlap_matrix(words), n_words=4, max_overlap=1, max_yields=1000)

for lex, llex in zip(lexicon_generator_1, lexicon_generator_2):
    print(lex, lex.info["cumulative_overlap"], llex, llex.info["cumulative_overlap"])

# Tutorial 2

We will load the words and generate a Lexicon with minimal feature overlap between the words. Then, we will follow up with the 2 main ways we can generate random streams, the `Word`-based and the `Syllable`-based stream generation.

First, we load the words from tutorial 1.

In [3]:
from arc import load_words

words = load_words("arc_results/arc_Words.json")
print(words)

beːhoːzuː|biːʃøːheː|buːsaːhøː|byːseːhiː|bøːseːhuː|deːfaːhiː|deːhoːfaː|deːhoːfiː|doːhuːfaː|faːhoːtuː|... (100 elements total)


Great, now we generate minimum-overlap lexicons with 4 words each. By default, the generator will yield 10 `Lexicon`s max. Let's generate 20 and print some info.

In [9]:
from arc.data import make_lexicon_generator

lexicon_generator = make_lexicon_generator(words, n_words=5, max_yields=20)

for lex in lexicon_generator:
    print(lex, lex.info)

Output()

heːvaːdoːtiːhuːfyːzyːbeːhoːfoːniːkuːɡaːsiːmɛː {'cumulative_overlap': 5.0}
heːvaːdoːdeːhoːfiːfaːkoːreːfoːniːkaːʃiːpeːhuː {'cumulative_overlap': 5.0}
beːhoːzuːbiːʃøːheːhuːfiːtɛːluːɡaːfyːʃaːmuːkɛː {'cumulative_overlap': 5.0}
heːvaːdoːdeːhoːfiːfaːkoːreːfoːriːkɛːʃiːpeːhuː {'cumulative_overlap': 5.0}
beːhoːzuːbiːʃøːheːhuːfiːdoːluːɡaːfyːʃaːbyːhiː {'cumulative_overlap': 5.0}
beːhoːzuːheːvaːdoːbyːseːhiːtoːfyːhøːɡiːroːfaː {'cumulative_overlap': 5.0}
deːhoːfaːhuːfiːtyːkaːsuːmyːfoːniːkuːfuːɡaːroː {'cumulative_overlap': 5.0}
huːfiːtyːfaːnyːkoːzyːbeːhoːluːɡaːfyːɡiːʃoːmyː {'cumulative_overlap': 5.0}
deːhoːfaːhøːtoːfiːfoːriːkɛːfuːɡaːnɛːʃiːpeːhuː {'cumulative_overlap': 5.0}
heːvaːdoːhøːtoːfiːfaːnyːkoːtyːhoːfuːʃiːpeːhuː {'cumulative_overlap': 5.0}
beːhoːzuːheːvaːdoːbyːseːhiːtoːfyːhøːɡiːroːfaː {'cumulative_overlap': 5.0}
heːvaːdoːdeːhoːfiːfaːkoːreːtoːfyːhøːfuːniːkaː {'cumulative_overlap': 5.0}
beːhoːzuːheːvaːdoːbyːseːhiːtoːfyːhøːɡiːroːfaː {'cumulative_overlap': 5.0}
deːhoːfaːhøːtoːfiːfoːriːkɛːfuːɡaːnɛːʃi